## CÓDIGO DE OTIMIZAÇÃO PARA O PROJETO FISH
O código tem como objetivo definir quais são os lotes à serem vendidos na semana de maneira que o lucro seja o maior possível.

### Função objetivo
A função objetivo atual é definida como **lucro = receita - custos**, onde:<br>
- **Receita**: Calculada como o produto do **preço do dia** pela **biomassa total** do lote, variando conforme o dia da semana determinado para a venda.<br>
- **Custos**: São a soma dos **custos totais** do lote e do **custo diário acumulado**, que aumenta proporcionalmente ao número de dias em que o lote é mantido antes de ser vendido.

### Dados utilizados
Os dados utilizados no projeto estão nos arquivos **`dados_lote-fish`** e **`param_otm`**. O primeiro contém informações dos lotes, como identificação, custos, biomassa e outros atributos importantes. Já o segundo armazena parâmetros de otimização, como limite máximo de vendas, preços por dia da semana e requisitos necessários para validar uma venda. Esses arquivos fornecem os insumos para o processo de otimização.

### Regras / Restrições
As regras atuais definidas para a venda dos lotes são:
- Um lote pode ser vendido apenas uma vez.
- Um lote não pode ter a data de venda anterior à data de movimentação
- O número máximo de lotes que podem ser vendidos num mesmo dia.
- O número mínimo de lotes que podem ser vendidos num mesmo dia.
- O mínimo de dias que um lote deve ter para venda
- O número máximo de dias que um lote pode ter sem estar vendido

### Saída / Retorno

Quando executado, o código gera um arquivo **CSV** nomeado **`output.csv`**. Este arquivo contém os lotes selecionados para venda na semana, junto com informações detalhadas, como:  
- **Identificação do lote**  
- **Data da venda** (dia da semana escolhido)  
- **Custos totais**
- **Biomassa total do lote**  
- **Lucro gerado pela venda**  
- Outros **atributos relevantes para análise e tomada de decisão**  

Esse arquivo pode ser utilizado para análise operacional e otimização contínua.

### Importação de bibliotecas
Realiza a importação das bibliotecas. A biblioteca pandas é utilizada para manipulação de dados, leitura dos dados em CSV e geração de arquivo CSV com os resultados da otimização. São utilizados módulos da biblioteca ortools para os cálculos de otimização linear.

In [440]:
#Instalação do pacote de otimização, principalmente para google colab
#!pip install ortools

In [441]:
import pandas as pd
from datetime import datetime, timedelta
#Pacotes de otimização
from ortools.linear_solver import pywraplp
from ortools.linear_solver.pywraplp import Variable
#Pacote para data handling da otimização
from typing import Dict, Set, Tuple, Union, List, Any
from collections import defaultdict
#Biblioteca para formatação de moeda
from babel.numbers import format_currency
#Biblioteca para geração de valores aleatórios
import random  
#Teste de interface
from ipywidgets import widgets, interact

### Declaração de variáveis
Declaração e tipagem das variáveis que serão utilizadas no projeto.

In [442]:
#Criação de um Dictonary, que tem chave o n° do Lote e como valor outro Dictionary
lotes_info = {}
lotes_param = {}
venda_lote = {}
venda_lote_dia = {}
dias = range(7)
meses_alta = []
objective: str = "lucro"
solver = None
solverParams = None

### Configuração de solver
Solver ou resolvedor que é criado para encontrar a solução de problemas matemáticos complexos. Nessa configuração ele possui uma tolerância padrão do tipo float e valor de 0,001 e um tempo limite de execução de 60.000ms (60s).

In [443]:
def config_solver(gap_limit: float = 1e-3, time_limit: int = 60000 ):    
        global solver, solverParams    

        solver = pywraplp.Solver('hello_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
        solver.EnableOutput()
        solver.SetTimeLimit(time_limit)
        solverParams = pywraplp.MPSolverParameters()
        solverParams.SetDoubleParam(solverParams.RELATIVE_MIP_GAP, gap_limit)

        return solver, solverParams

### Leitura de arquivos
É realizada a leitura dos arquivos CSV de dados e parâmetros e suas informações são convertidas para um DataFrame.

In [444]:
dados_lotes = pd.read_csv('dados_lote_fish.csv', delimiter=';')
param_otm = pd.read_csv('param_otm.csv', delimiter=';')

### Armazenamento de informações
Essa função percorre todas as linhas do DataFrame "todos_abatedouros" e armazena o conteúdo de cada linha nas suas respectivas variáveis.

In [445]:
def get_values():
    global lotes_info, dias

    for _, row in dados_lotes.iterrows():
        lote = str(row.id_lote)

        #Criação do Dictionary interno que contém as informações do lote
        if lote not in lotes_info:
            lotes_info[lote] = {}

        #Grava data de movimentação
        lotes_info[lote]["data_movimentacao"] = row.data_movimentacao
        #Grava o dia da semana de movimentação
        data_movimentacao = datetime.strptime(lotes_info[lote]["data_movimentacao"], "%Y-%m-%d %H:%M:%S")  
        lotes_info[lote]["dia_semana_movimentacao"] = data_movimentacao.weekday()
        #Gravação de outras informações do lote
        lotes_info[lote]["tanque_origem"] = row.tanque_origem
        lotes_info[lote]["especie"] = row.especie
        lotes_info[lote]["n_dias"] = row.n_dias
        lotes_info[lote]["peso"] = row.peso
        lotes_info[lote]["conversao"] = row.conversao
        lotes_info[lote]["biomassa"] = row.biomassa
        lotes_info[lote]["biomassa_esperada_total"] = row.biomassa_esperada_total
        lotes_info[lote]["custos_totais"] = row.custos_totais
        #Teste custo diário considerando custos totais e n° dias
        lotes_info[lote]["custo_diario"] = (row.custos_totais / row.n_dias)

In [446]:
#Exibição dos lotes para teste
"""
global lotes_info

get_values()

import json
for lote in lotes_info:
    #print(f"Lote: {lote} {lotes_info[lote]}")
    print(json.dumps(lotes_info, indent=4))    
"""

'\nglobal lotes_info\n\nget_values()\n\nimport json\nfor lote in lotes_info:\n    #print(f"Lote: {lote} {lotes_info[lote]}")\n    print(json.dumps(lotes_info, indent=4))    \n'

### Parâmetros de cálculo
Nessa função são declarados os parâmetros para o calculo de otimização linear.

In [447]:
def get_params():
    global lotes_param, dias, meses_alta

    #Percorre todas as linhas do arquivo
    for _,row in param_otm.iterrows():
        dia = row.dia_semana
        #Criação do Dictionary interno que contém as informações para o dia
        if dia not in lotes_param:
            lotes_param[dia] = {}
        #Gravação das informções
        lotes_param[dia]["preco_dia"] = row.preco_dia
        lotes_param[dia]["maximo_venda_dia"] = row.maximo_venda_dia        
        lotes_param[dia]["minimo_venda_dia"] = row.minimo_venda_dia
        lotes_param[dia]["minimo_dias"] = row.minimo_dias
        lotes_param[dia]["maximo_dias"] = row.maximo_dias   
        lotes_param[dia]["peso_minimo"] = row.peso_minimo  
        lotes_param[dia]["peso_maximo"] = row.peso_maximo  
        lotes_param[dia]["taxa_min_conversao"] = row.taxa_min_conversao
        lotes_param[dia]["biomassa_total"] = row.biomassa_total   
        lotes_param[dia]["mes_alta_preco"] = row.mes_alta_preco 
    #Definição dos meses de alta no preço, os meses são definidos no CSV e portanto padrão para todos os lotes
    meses = [mes.strip() for mes in lotes_param[0]["mes_alta_preco"].split(",")]
    for mes in meses:
        if mes.isdigit():
            meses_alta.append(int(mes))
    print(meses_alta[2])

### Variáveis de otimização
A função configura variáveis para utilização no processo de otimização linear.

In [448]:
def set_variables_optimizer():
    global solver, lotes_info, venda_lote, venda_lote_dia, dias

    for lote in lotes_info:
        #Determina se o lote será ou não vendido
        venda_lote[lote] = solver.BoolVar(f'venda_lote_{lote}')
        for dia in dias:
            #Determina se o lote será ou não vendido no dia selecionado
            venda_lote_dia[lote, dia] = solver.BoolVar(f'venda_lote_{lote}_dia{dia}')

### Objetivo
Definição do objetivo da maximização do lucro.

In [449]:
def get_objective_value():
    global venda_lote_dia, lotes_info, lotes_param, solver, dias, objective

    objective = solver.Objective()

    for lote in lotes_info:        
        for dia in dias:
            #Variáveis da função objetivo       
            receita = lotes_param[dia]["preco_dia"]  * lotes_info[lote]["biomassa"]
            custo = lotes_info[lote]["custos_totais"] + (lotes_info[lote]["custo_diario"] * dia)
            #Função objetivo
            lucro = receita - custo         
            objective.SetCoefficient(venda_lote_dia[lote, dia], lucro)            
    objective.SetMaximization()

### Regras da otimização
A função define regras da otimização linear para garantir o melhor momento de venda do lote.

In [450]:
def optimization_rules():
    global solver, lotes_info, lotes_param, venda_lote, venda_lote_dia, dias, meses_alta
    
    #Limites definidos por lotes
    for lote in lotes_info:
        #Garante que um lote seja vendido uma única vez
        solver.Add(sum(venda_lote_dia[lote, dia] for dia in dias) == venda_lote[lote])
        #Garante que um lote não tenha uma data de venda anterior à data de movimentação
        dia_minimo = lotes_info[lote]["dia_semana_movimentacao"]         
        solver.Add(sum(venda_lote_dia[lote, dia] for dia in range(dia_minimo - 1)) == 0)         
            
        for dia in dias:           
            #O lote não pode ser vendido antes do número mínimo de dias
            n_dias_atual = lotes_info[lote]["n_dias"] + dia 
            if(n_dias_atual < lotes_param[dia]["minimo_dias"]):
                solver.Add(venda_lote_dia[lote,dia] == 0)
                
            #O lote deve ser vendido na semana ao atingir ou ultrapassar o número máximo de dias
            if(n_dias_atual >= lotes_param[dia]["maximo_dias"]):
                solver.Add(sum(venda_lote_dia[lote, d] for d in dias) == 1)

            #Definição mês de alta
            mes_atual = datetime.now().month
            peso_minimo = lotes_param[dia]["peso_minimo"]
            if mes_atual in meses_alta:
                peso_minimo = lotes_param[dia]["peso_minimo"] * 0.7                

            #O lote deve atender o peso médio mínimo exigido
            if(lotes_info[lote]["peso"] < peso_minimo):
                solver.Add(venda_lote_dia[lote, dia] == 0)  

            #Lotes com conversão alimentar acima do limite devem ser vendidos
            if(lotes_info[lote]["conversao"] >= lotes_param[dia]["taxa_min_conversao"]):
                solver.Add(sum(venda_lote_dia[lote, d] for d in dias) == 1)             

    #Limites definidos por dia    
    for dia in dias:
        #Quantidade máxima de lotes vendidos no dia
        solver.Add(sum(venda_lote_dia[lote, dia] for lote in lotes_info) <= lotes_param[dia]["maximo_venda_dia"])
        #Quantidade mínima de lotes vendidos no dia
        solver.Add(sum(venda_lote_dia[lote, dia] for lote in lotes_info) >= lotes_param[dia]["minimo_venda_dia"])
        #Define um limite de biomassa por dia   
        soma_biomassa = sum(venda_lote_dia[lote,dia] * lotes_info[lote]["biomassa"] for lote in lotes_info)     
        solver.Add(soma_biomassa <= lotes_param[dia]["biomassa_total"])


### Execução do solver
A função realiza os cálculos de otimização linear

In [451]:
def solver_resolver():
    global solver, venda_lote, venda_lote_dia, solverParams, lotes_info, dias
    infos = []
    #Execução do solver
    status = solver.Solve(solverParams)
    sem = ("Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo")  
    #Verificação de resultados ótimos ou viáveis
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f'Funcao objetivo ={solver.Objective().Value()}')
        print(solver.WallTime())
        total_lucro = 0
        for lote in lotes_info:
            for dia in dias:
                #Trabalha apenas com os lotes selecionados para venda
                if venda_lote_dia[lote, dia].solution_value() == 1:
                    #Manipulação de data para definição do dia da venda
                    data_movimentacao = datetime.strptime(lotes_info[lote]["data_movimentacao"], "%Y-%m-%d %H:%M:%S")  
                    dia_semana = lotes_info[lote]["dia_semana_movimentacao"]                               
                    data_venda = data_movimentacao + timedelta(dia - dia_semana) 
                    #Exibição dos lotes selecionados
                    print(f'Lote {lote} será vendido em {data_venda}, {sem[dia]}.')                
                    lucro = round((lotes_param[dia]["preco_dia"] * lotes_info[lote]['biomassa']) - lotes_info[lote]['custos_totais'] - (lotes_info[lote]["custo_diario"] * dia),2)
                    #Armazenamento de informações do lote selecionado
                    infos.append([
                        lote,
                        data_venda,
                        lotes_info[lote]["tanque_origem"],
                        lotes_info[lote]["n_dias"],
                        lotes_info[lote]["especie"],
                        lotes_info[lote]["custo_diario"],
                        lotes_info[lote]["custos_totais"] + (lotes_info[lote]["custo_diario"]*dia),
                        lotes_info[lote]["biomassa"],
                        lotes_info[lote]["biomassa_esperada_total"],
                        lucro
                    ])
                    total_lucro += lucro        
        print(f'Lucro bruto: {format_currency(total_lucro, "BRL", locale="pt_BR")}')
        print(f'Lucro líquido estimado: {format_currency((total_lucro * 0.6), "BRL", locale="pt_BR")}')
        return infos
    else:
        print('Nao resolveu o problema!')
        return None

In [452]:
"""def show_infos(lote):
    lote_info = df[df["Lote"] == lote]
    display(lote_info.style.set_table_styles(
        [{'selector': 'th', 'props': [('font-weight', 'bold'), ('background-color', '#b352ff')]}]
    ))  """  

'def show_infos(lote):\n    lote_info = df[df["Lote"] == lote]\n    display(lote_info.style.set_table_styles(\n        [{\'selector\': \'th\', \'props\': [(\'font-weight\', \'bold\'), (\'background-color\', \'#b352ff\')]}]\n    ))  '

### DropDown
Função destinada à configuração e exibição de um DropDown com os lotes selecionados para venda, contendo informações mais detalhadas.

In [453]:
def show_dropdown(df):
    def show_infos(lote):
        lote_info = df[df["ID LOTE"] == lote]
        display(lote_info.style.set_table_styles(
            [{'selector': 'th', 'props': [('font-weight', 'bold'), ('background-color', '#003333'), ('color','white'), ('text-align','center'), ('font-size','12px')]},
            {'selector': 'td', 'props': [('background-color', '#004c4c'), ('text-align','center'), ('color','white'), ('font-size','12px')]},
            ]
        ))   

    # Criar dropdown com opções de lotes
    dropdown = widgets.Dropdown(
        options=df["ID LOTE"].tolist(),
        description='Selecione o Lote:',
        style = {"description_width":"120px"},
        layout = {"width":"200px"},          
    )
    
    interact(show_infos, lote = dropdown) 


### Função: optmize
Esta função funciona por meio da chamada de funções variáveis, parâmetros e regras de otimização, através do solver faz o cálculo de otimização e gera o output.csv por meio da utilização da biblioteca pandas.

In [454]:
def optimize():
        #Chamada e execução das funções
        config_solver()
        get_values()        
        get_params()                
        set_variables_optimizer()
        get_objective_value()
        optimization_rules()        

        #Chamada para execução do solver
        if not isinstance(f := solver_resolver(), type(None)):
            column_names = [
                "ID LOTE","DATA VENDA","TANQUE ORIGEM", "N DIAS", "ESPECIE", "CUSTO DIARIO", "CUSTOS TOTAIS", "BIOMASSA", "BIOMASSA ESPERADA TOTAL", "LUCRO"
            ]
            #Gravação dos dados no DataFrame e geração do CSV
            df = pd.DataFrame(f,columns=column_names) 
            df = df.sort_values(by="DATA VENDA")
            df.to_csv("output.csv", sep=";", decimal=",", columns=column_names, index=False)  
            show_dropdown(df)           

### Execução do programa
Chamada da função Optimize, que realiza o processo de otimização e finzaliza gerando o arquivo CSV com os resultados.

In [455]:
optimize()

3
Funcao objetivo =205290.4013764586
396
Lote 5709 será vendido em 2022-01-07 00:00:00, Sexta.
Lote 6173 será vendido em 2022-01-03 00:00:00, Segunda.
Lote 6036 será vendido em 2022-01-03 00:00:00, Segunda.
Lote 5948 será vendido em 2022-01-03 00:00:00, Segunda.
Lote 5821 será vendido em 2022-01-05 00:00:00, Quarta.
Lote 5739 será vendido em 2022-01-06 00:00:00, Quinta.
Lote 6033 será vendido em 2022-01-04 00:00:00, Terça.
Lote 6035 será vendido em 2022-01-04 00:00:00, Terça.
Lote 6024 será vendido em 2022-01-06 00:00:00, Quinta.
Lote 6008 será vendido em 2022-01-05 00:00:00, Quarta.
Lote 5792 será vendido em 2022-01-06 00:00:00, Quinta.
Lote 6236 será vendido em 2022-01-06 00:00:00, Quinta.
Lote 6037 será vendido em 2022-01-04 00:00:00, Terça.
Lote 6030 será vendido em 2022-01-05 00:00:00, Quarta.
Lote 5793 será vendido em 2022-01-07 00:00:00, Sexta.
Lote 6206 será vendido em 2022-01-10 00:00:00, Segunda.
Lote 6185 será vendido em 2022-01-11 00:00:00, Terça.
Lote 6018 será vendido em 

interactive(children=(Dropdown(description='Selecione o Lote:', layout=Layout(width='200px'), options=('6173',…

In [456]:
#RASCUNHOS E TESTES

# Dados fictícios do lote
data = {
    "Lote": ["Lote 1", "Lote 2", "Lote 3"],
    "Data da Venda": ["2025-02-03", "2025-01-29", "2025-02-01"],
    "Lucro": [1500.75, 2000.25, 1800.50],
    "Biomassa": [1200, 1400, 1300],
    "Custos Totais": [500, 600, 550]
}
df = pd.DataFrame(data)

# Função para exibir informações do lote
def exibir_informacoes(lote):
    lote_info = df[df["Lote"] == lote]
    display(lote_info.style.set_table_styles(
        [{'selector': 'th', 'props': [('font-weight', 'bold'), ('background-color', '#b352ff')]}]
    ))

# Criar dropdown com opções de lotes
dropdown = widgets.Dropdown(
    options=df["Lote"].tolist(),
    description='Lotes:',
    value="Lote 1"
)

# Interatividade
interact(exibir_informacoes, lote=dropdown)

interactive(children=(Dropdown(description='Lotes:', options=('Lote 1', 'Lote 2', 'Lote 3'), value='Lote 1'), …

<function __main__.exibir_informacoes(lote)>